In [58]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST 데이터셋 
train_data = datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

In [35]:
!pip install wandb

In [59]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [60]:
args = dict(
    total_epochs=3,
    batch_size=128,
    learning_rate=0.005,
    num_classes=10,
    hidden_size=100)

In [61]:
wandb.init(project='fast_campus_deep_learning(Pytorch)')
wandb.config.update(args)

CE loss,0.1303
Current Epoch,2
test accuracy,0.97288


CE loss,█▃▃▃▅▃▃▂▄▂▂▃▂▂▂▂▁▂▃▃▂▁▂▁▂▂▃▁▁▁▃▁▁▂▂▂▂▃▃▂
Current Epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅█████████████
test accuracy,▄▁▂▃▅▇█


In [62]:
# Data loader
train_loader = DataLoader(train_data, batch_size=args['batch_size'], shuffle=True)
test_loader = DataLoader(test_data, batch_size=args['batch_size'], shuffle=False)

In [63]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.relu(out)
        out = self.mlp2(out)
        
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=args['hidden_size'], num_classes=args['num_classes']).to(device)

In [66]:
CELoss = nn.CrossEntropyLoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=args['learning_rate']) # 최적화 알고리즘 class 선언
wandb.watch(model)

[]

In [67]:
def test_model():

    with torch.no_grad():
        correct = 0
        total = 0

        for i, (images, labels) in enumerate(test_loader):
            print('i:',i)
            images_flatten = images.reshape(-1, 28*28*1).to(device)
            labels = labels.to(device)

            outputs = model(images_flatten)
            _ , predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            log_dict={"test accuracy":correct/total}
            wandb.log(log_dict)

In [68]:
train_model(args)

Epoch [1/3], Loss: 0.1965
Epoch [2/3], Loss: 0.0748
Epoch [3/3], Loss: 0.0145


In [71]:
def test_model():
    # 학습이 끝난 후 모델 성능 테스트
    # test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해
    
    images_ex = []
    with torch.no_grad(): # gradient 계산하지 않도록 하는 코드
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images_flatten = images.reshape(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images_flatten)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if (i+1)%10 == 0: # 10번 iteration을 돌 때 마다
                images_ex.append(wandb.Image(
                    images[0], caption="predicted: {} label: {}".format(predicted[0].item(), labels[0])))
                log_dict={"test accuracy":correct/total}
                wandb.log(log_dict)
                
        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [72]:
test_model()

Accuracy of the network on the 10000 test images: 96.95 %
